In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bone-fracture-detection-computer-vision-project:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4296838%2F7391382%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240811%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240811T181410Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D55135b7e6ad141faefafe510657bceceb42eca6737fc52043406135ad5bf956ff34ace16f87ec6a2678dc2be09858dc6dfd53639a21d824362f62e026e968400ffd9522a0e3f08c3c780a2cd0ea5c7328b0736d902c611aef58b0d9c83b6e5c622e50cea4b58e8929361c00ab4b7b3d4aa6bf6f884db110010d7a585f24cb34cdc1183e9d49a7c49cf82065d12b360278ab2b5d859e69f5ed54703217d528d7b5d0ce78fe9d0454ec807eefbf997441c1e748ef4fc0faf4e4b307bbe1850b01109d24083e8f69a348a4d0b68db2a2ad1d70afed69249a6df1ce6791e22628474f8695f1860b47b38cbd09f287a6204bfd11806880c9d5f1605810ea727d582d6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [3]:
!conda install -y numpy

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.7.1




In [ ]:
conda install -c pytorch pytorch

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from torch import nn
import albumentations as A
# import torchsummary
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random

torch.cuda.empty_cache()

BS=16
LR=0.00005
epochs=20
IS=256
D='cuda' if torch.cuda.is_available() else 'cpu'

num_classes = 7
classes=['elbow positive', 'fingers positive', 'forearm fracture', 'humerus fracture', 'humerus', 'shoulder fracture', 'wrist positive']
c2l={k:v for k,v in list(zip(classes,list(range(num_classes))))}
l2c={v:k for k,v in c2l.items()}

dir_path='/kaggle/input/bone-fracture-detection-computer-vision-project'
train_dir_path='/kaggle/input/bone-fracture-detection-computer-vision-project/train'
train_img_paths = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/train/images'))
train_target_paths = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/train/labels'))

val_dir_path='/kaggle/input/bone-fracture-detection-computer-vision-project/valid'
val_img_paths = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/valid/images'))
val_target_paths = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/valid/labels'))

test_dir_path='/kaggle/input/bone-fracture-detection-computer-vision-project/test'
test_img_paths = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/test/images'))
test_target_paths = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/test/labels'))


In [ ]:
def unconvert(width, height, x, y, w, h):

    xmax = int((x*width) + (w * width)/2.0)
    xmin = int((x*width) - (w * width)/2.0)
    ymax = int((y*height) + (h * height)/2.0)
    ymin = int((y*height) - (h * height)/2.0)

    return xmin, ymin, xmax, ymax

In [ ]:
augs=A.Compose([
    A.Resize(IS,IS),
],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']), is_check_shapes=True)

In [ ]:
class FractureData(torch.utils.data.Dataset):

    def __init__(self, dir_path, img_paths, target_paths, augs=None):
        self.dir_path=dir_path
        self.img_paths=img_paths
        self.target_paths=target_paths
        self.augs=augs

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self,idx):
        ip=os.path.join(self.dir_path,'images',self.img_paths[idx])
        tp=os.path.join(self.dir_path,'labels',self.target_paths[idx])

        image=cv2.imread(ip)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        H,W,_=image.shape

        file=open(tp,'r')
        target=list(map(float,file.read().split()))

        try:
            label=[target.pop(0)]
            bbox=[]
            i=0
            while i<len(target):
                x,y,w,h=target[i:i+4]
                bbox.append([*unconvert(W,H,x,y,w,h)])
                i+=4
            label=label*len(bbox)

            if self.augs!=None:
                data=self.augs(image=image,bboxes=bbox,class_labels=['None']*len(label))
                image=data['image']
                bbox=data['bboxes']
        except:
            if idx+1<len(self.img_paths):
                return self.__getitem__(idx+1)
            else:
                return self.__getitem__(0)

        image=torch.Tensor(np.transpose(image,(2,0,1)))/255.0
        bbox=torch.Tensor(bbox).long()
        label=torch.Tensor(label).long()

        annot={'boxes':bbox,'labels':label}

        return image, annot

    def collate_fn(self,batch):
        return tuple(zip(*batch))

In [ ]:
trainset=FractureData(train_dir_path, train_img_paths, train_target_paths, augs)
valset=FractureData(val_dir_path, val_img_paths, val_target_paths, augs)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=BS,collate_fn=trainset.collate_fn)
valloader=torch.utils.data.DataLoader(valset,batch_size=BS,collate_fn=valset.collate_fn)


testset=FractureData(test_dir_path, test_img_paths, test_target_paths, augs)
testloader=torch.utils.data.DataLoader(testset,batch_size=BS,collate_fn=valset.collate_fn)
print(f'Training Data:- {len(testset)} images divided into {len(testloader)} batches')

In [ ]:
for image,target in testloader:
    break

#### Model

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model=torchvision.models.detection.fasterrcnn_resnet50_fpn(preTrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.to(D);

In [ ]:

model.load_state_dict(torch.load("C:\\Users\\FPCC\\Desktop\\Bone Fracture Detection Using RCNN\\model.pt"))

In [ ]:

def f(img):
    test_img,test_tar=img
    test_imgo=test_img.permute(1, 2, 0).numpy()
    plt.imshow(test_imgo)
    plt.title('Original Image')
    plt.show()
    model.eval()
    pred=model(test_img.unsqueeze(0).to(D))
# pred=torchvision.ops.nms(pred[0]['boxes'].detach(),pred[0]['scores'].detach(),0.02)
    xmin,ymin,xmax,ymax=pred[0]['boxes'][0].detach().cpu().long().tolist()
    label=pred[0]['labels'][0].item()
    Txmin,Tymin,Txmax,Tymax=test_tar['boxes'][0].tolist()
    image=cv2.rectangle(test_img.permute(1,2,0).numpy(),(xmin,ymin),(xmax,ymax),(255,0,0),2)
    plt.imshow(image)
    plt.title('predicted Image' )
    plt.show()


In [ ]:
idx=290
img=testset[idx]
f(img)

In [ ]:
from PIL import Image
image = Image.open('/content/test7.png')
import numpy as np
image = np.array(image)
image = torch.Tensor(np.transpose(image,(2,0,1)))/255.0
test_imgo=image.permute(1, 2, 0).numpy()